<a href="https://colab.research.google.com/github/jiztastamablastamarang/TwitterSturm/blob/main/TwitterSoldier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Bot##

###Modules###

In [6]:
!pip install -U -q selenium
!pip install -U -q snscrape

!apt-get -qq update
!apt install -q chromium-chromedriver
!dpkg --configure -a

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.9 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
selenium 4.1.3 requires urllib3[secure,socks]~=1.26, but you have urllib3 1.25.11 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
Reading package lists...
Building dependency tree...
Reading state information...
chromium-chromedriver is already the newest version (99.0.4844.84-0ubuntu0.18.04.1).
0 upgraded, 0 newly installed, 0 to remove and 82 not upgraded.


In [7]:
import pandas as pd
import requests
import os
import glob
import random
import shutil
import time
from datetime import datetime, timedelta
from tqdm import tqdm
from collections import UserList

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

from IPython.display import Image
import snscrape.modules.twitter as sns


In [8]:
def screenshot(webdriver):
  t=3
  time.sleep(t)
  webdriver.fullscreen_window()
  img = webdriver.get_screenshot_as_png()
  return Image(img)

###Class###

In [33]:
class TwitterSoldier:

    def __init__(self, phone, password, username):
      self.email = phone
      self.password = password
      self.nick = username
      t = 10
      options = Options()
      options = webdriver.ChromeOptions()
      options.add_argument("--start-maximized")
      options.add_argument('--headless')
      options.add_argument('--no-sandbox')
      options.add_argument('--disable-dev-shm-usage')
      options.add_argument("--window-size=3920,2080")
      # options.add_argument("--blink-settings=imagesEnabled=false") # disable images
      self.driver = webdriver.Chrome(executable_path="chromedriver", options=options)
      self.waiter = WebDriverWait(self.driver, t)
      self.action = ActionChains(self.driver)
      

    def login(self):
      messedge = "We've texted you a login verification code"
      driver = self.driver
      wait = self.waiter
      driver.get("https://twitter.com/login")
      time.sleep(3)

      def enter(text):
        field = wait.until(EC.presence_of_element_located((By.TAG_NAME, "input")))
        field.send_keys(text)
        field.send_keys(Keys.ENTER)
        time.sleep(3)

      enter(self.email)
      enter(self.password)

      if driver.page_source.count(messedge):
        print(messedge)
        text = input()
        enter(text)

      try:
        enter(self.nick)
      except:
        pass

      return screenshot(driver)


    def reply_status(self, url, status, image):
        driver = self.driver
        wait = self.waiter
        action = self.action 
        driver.get(url)
        list = []
        try:
          reply = wait.until(EC.presence_of_element_located((By.XPATH, "//div[@data-contents='true']")))
          reply.send_keys(status)
          if image:
            media = driver.find_element(By.XPATH, "//input[@data-testid='fileInput']")
            media.send_keys(image)
          button = wait.until(EC.element_to_be_clickable((By.XPATH, "//div[@data-testid='tweetButtonInline']")))
          button.click()
        except:
          print(f" error sending reply to status: {url}")
          return url, screenshot(driver)


    def get_tweet_from_user(self, user, query, start):
      tweets = sns.TwitterSearchScraper(f"from:{user} since:{start}").get_items()
      for i, tweet in enumerate(tweets):
        if tweet.content.lower().count(query.lower()) and i<100:
          return tweet.url
      return tweets[0].url

    
    def get_tweets_from_query(self, query, start=None, end=None, days=1):
      tweets = []
      if not all([start, end]):
        end = datetime.now().strftime("%Y-%m-%d")
        start = (datetime.now() - timedelta(days=days)).strftime("%Y-%m-%d")
      for i, tweet in enumerate(sns.TwitterSearchScraper(f"{query} since:{start} until:{end}").get_items()):
          tweets.append({"date": tweet.date, 
                         "username": tweet.username, 
                         "url": tweet.url, 
                         "content": tweet.content})
        # tweets.append({"date": tweet.date, 
        #                "username": tweet.user.username, 
        #                "followers": tweet.user.followersCount,
        #                "url": tweet.url, "content": tweet.content
        #                })
      return tweets


    # def user_followers(self, user):
    #   driver = self.driver
    #   wait = self.waiter
    #   driver.get(f"https://twitter.com/{user}/")
    #   followers =  wait.until(EC.presence_of_element_located((By.XPATH, f"//a[@href='/{user}/followers']")))
    #   return followers.text.split()[0].replace(",", "").replace(".", "").replace("K", "00").replace("M", "00000")


##Docs##
Гуглотаблиці списком користувачей, контентом, директорія з фото

In [12]:
USERS = "https://docs.google.com/spreadsheets/d/1uUF6WGtFl-WBPe_oLdk4gdgFp09aPOAekBWmHdIUnC0/edit#gid=0" #повний список сенаторів
CONTENT = "https://docs.google.com/spreadsheets/d/1dsdTocyIgQpJYrBLnjxBl9e3Bdw65vAWmpBPorzLlrI/edit#gid=0"# коменти
IMGS = "https://drive.google.com/drive/folders/1lVzCxEtXHkorxiF9ac9LjkCGq1wOlVc7" # директорія з фото

###Images###

In [13]:
folder_id = IMGS.split("/")[-1]
images = []

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

path = "./temp"
try: shutil.rmtree(path)
except: pass

try: os.makedirs(path)
except: pass

file_list = drive.ListFile(
    {'q': "'{}' in parents".format(folder_id)}).GetList()

for image in file_list:
    fname = os.path.join(path, image["title"])
    new_file = drive.CreateFile({"id": image["id"]})
    new_file.GetContentFile(fname)
    print("uploading: " + image["title"])
    images.append(os.path.join("/content", "temp", image["title"] ))

uploading: IMG_6097.JPG
uploading: IMG_6096.JPG
uploading: IMG_6095.PNG
uploading: IMG_6094.PNG
uploading: IMG_6093.PNG
uploading: IMG_6092.JPG
uploading: IMG_6091.JPG
uploading: users.xlsx
uploading: download.jpg


###Users###

In [15]:
def load_doc(url):
  return pd.read_csv(url.replace("/edit#gid=", "/export?format=csv&gid="), on_bad_lines="skip")

In [16]:
users = load_doc(USERS)
try:
  users["TwitterNickname"] = users["Twitter"].str.replace("https://twitter.com/", "", regex=False).str.replace("@", "", regex=False)
except:
  pass
users.dropna(subset=["TwitterNickname"], inplace=True)
users = UserList(users["TwitterNickname"])
users

['SenShelby', 'Ttuberville', 'RepJerryCarl', 'RepBarryMoore', 'RepMikeRogers', 'Robert_Aderholt', 'RepMoBrooks', 'USRepGaryPalmer', 'RepTerriSewell', 'SenDanSullivan', 'lisamurkowski', 'RepDonYoung', 'RepAmata', 'SenMarkKelly', 'SenatorSinema', 'TomOHalleran', 'RepKirkPatrick', 'RepRaulGrijalva', 'RepGosar', 'AndyBiggs4AZ', 'RepDavid', 'RepRubenGallego', 'RepDLesko', 'RepGregStanton', 'SenTomCotton', 'JohnBoozman', 'RepRickCrawford', 'RepFrenchHill', 'rep_stevewomack', 'RepWesterman', 'SenFeinstein', 'KamalaHarris', 'RepLaMalfa', 'RepHuffman', 'RepGaramendi', 'RepMcClintock', 'RepThompson', 'DorisMatsui', 'RepBera', 'JayObernolte', 'RepMcNerney', 'RepJoshHarder', 'RepDeSaulnier', 'SpeakerPelosi', 'RepBarbaraLee', 'RepSpeier', 'RepSwalwell', 'RepJimCosta', 'RoKhannaUSA', 'RepAnnaEshoo', 'RepZoeLofgren', 'JimmyPanetta', 'RepDavidValadao', 'GOPLeader', 'CarbajalSalud', 'MikeGarcia2020', 'RepBrownley', 'RepJudyChu', 'RepAdamSchiff', 'RepCardenas', 'BradSherman', 'RepPeteAguilar', 'gracenap

###Content###

In [17]:
content = load_doc(CONTENT)["Messedge"].to_list()
load_doc(CONTENT)

,Messedge,Lengh
0,The war in Ukraine looms dangerously over Euro...,87


##Credentials & login##

In [ ]:
PHONE = ""
USERNAME = ""
PASSWORD = ""

bot = TwitterSoldier(PHONE, PASSWORD, USERNAME)
bot.login()

In [ ]:
# with tqdm(total=len(tweets)) as progress:
#   for tweet in tweets:
#     user = tweet["username"]
#     print(bot.user_followers(user))
#     progress.update()

##Main##

In [ ]:
# tweets = bot.get_tweets_from_query("Ukraine", days=1)
# tweets = pd.read_excel("/content/drive/MyDrive/STANDU/users.xlsx")
# tweets = tweets["url"].to_list()
brands = ["Auchan"]
tweets = []
for brand in brands:
  temp = bot.get_tweets_from_query(brand, days=3)
  tweets = tweets + temp

In [35]:
status = random.choice(content)
media = random.choice(images)
bot.reply_status("https://twitter.com/jiztastamablast/status/1509713457754431490", status, media)

In [ ]:
false_shots = []

with tqdm(total=len(tweets)) as progress:
  for tweet in tweets:
    status = random.choice(content)
    media = random.choice(images)
    false_shots.append(bot.reply_status(tweet["url"], status, media))
    progress.update()
